# Feature Engineering

In [1]:
# Take the first n numbers from the upc as a string
# Take the upc length as a string
# Make departments all separate features and count the number of items purchased under each department
# Make broader departments such as Clothing for all Clothing
# Weekend vs Weekday indicator (1 and 0)
# Make null a binary indicator (1 and 0)
# Replace upc null with "no_upc"
# Replace finelinenumber null with "no_fineline"
# Ratio of total_scancount / num_unique_upc = avg_scancount_per_upc
# Ratio of total_scancount / num_unique_dept = avg_scancount_per_dept
# Ratio of num_unique_fineline / num_unique_dept = fineline_to_dept_ratio
# Ratio of num_unique_upc / num_unique_fineline = avg_upc_per_fineline (maybe)
# Ratio of num_unique_upc / num_unique_dept = avg_upc_per_dept